In [1]:
# part 1: 导入相关的 package
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from dataclasses import dataclass

import math

torch.manual_seed(1024)

In [ ]:
##2.GPT参数

In [ ]:
@dataclass
class GPTConfig:
    block_size: int = 512
    batch_size: int = 12
    n_layer: int = 12
    n_head: int = 12
    n_embd: int = 768 #hidden_dim, hidden_size
    droupout: float = 0.1
    head_size: int = n_embd//n_head
    # vocab_size 
    # gpt2 官方tokenizer
    vocab_size: int = 50257 